In [8]:
%matplotlib qt5
import cv2, time
import numpy as np
import argparse, pprint
from matplotlib import pyplot as plt

import pyrealsense2 as rs

from scripts.devices.d415_camera import CameraD415
# from src.VBOATS import VBOATS

# from scripts.devices.r200_camera import CameraR200

from scripts.hyutils.sys_utils import *
from scripts.hyutils.img_utils import *
from scripts.hyutils.debug_utils import *
from scripts.hyutils.plotting_utils import *
from scripts.hyutils.uv_mapping_utils import *

pp = pprint.PrettyPrinter(indent=4)

In [5]:
ctx = rs.context()
for d in ctx.devices:
    print ('Found device: ', \
    d.get_info(rs.camera_info.name), ' ', \
    d.get_info(rs.camera_info.serial_number))
dev = ctx.devices[0]


('Found device: ', u'Intel RealSense D415', ' ', u'847412062231')


In [14]:
cam.__del__()

[INFO] Closing CameraD415 object


In [11]:
cam = CameraD415(flag_save=False,use_statistics=False,fps=30, verbose=True)

('Found device: ', u'Intel RealSense D415', ' ', u'847412062231')
[INFO] CameraD415() -- Resetting Hardware...
[INFO] CameraD415() -- Initialization successful!!!


In [ ]:
vboat = VBOATS()

In [12]:
extr = cam.get_extrinsics()
intr = cam.get_intrinsics()

pp.pprint(extr)
pp.pprint(intr)

rotation: [0.999999, 0.000126028, 0.00157422, -0.000125856, 1, -0.000109517, -0.00157424, 0.000109319, 0.999999]
translation: [0.0147318, -6.67465e-05, -0.00013899]
{   u'color': width: 640, height: 480, ppx: 322.508, ppy: 234.413, fx: 614.156, fy: 613.426, model: Brown Conrady, coeffs: [0, 0, 0, 0, 0],
    u'depth': width: 640, height: 480, ppx: 319.742, ppy: 242.007, fx: 596.392, fy: 596.392, model: Brown Conrady, coeffs: [0, 0, 0, 0, 0]}


In [13]:
rgb, depth = cam.read()

tmp = depth/cam.dmax_avg
depth = np.uint8(tmp*255)


plt.figure(0)
plt.imshow(depth)
plt.show()

In [ ]:
vboat.pipeline(depth, threshU1=8,threshU2=20, threshV2=70)

In [ ]:
dispU1, dispU2, compU = vboat.umap_displays()
dispV1, dispV2, compV = vboat.vmap_displays()

plt.figure(2)
# plt.imshow(vboat.vmap_raw)
# plt.imshow(vboat.umap_raw)
plt.imshow(vboat.umap_processed)
plt.imshow(dispU2)
plt.show()

In [ ]:
display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)

for ob in vboat.obstacles:
    cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)

In [ ]:
def calculate_statistics(camera,duration=10.0):
    print("Calculating Average Max Disparity.....")
    t0 = time.time()
    sum, count = 0, 0
    while True:
        t1 = time.time()
        dt = t1 - t0
        if(dt<= duration):
            _, depth = camera.read()
            if(depth is None): continue
            sum += np.max(depth)
            count+=1
        else: break
    avg = sum / float(count)
    print("Average Max Disparity, Sum, Count: %.3f, %.3f, %d" % (avg,sum,count))
    return avg

In [ ]:
cam = CameraR200()

In [ ]:
intr,scale = cam.get_intrinsics()
extr, b = cam.get_extrinsics()

In [ ]:
avgDisparity = calculate_statistics(cam)

In [ ]:
print(scale,avgDisparity)

In [ ]:
rgb, depth = cam.read()
pplot(depth,"hey")
pplot(rgb,"rgb")

In [ ]:
tmp = depth/avgDisparity
tmp = np.uint8(tmp*255)
tmp = cv2.cvtColor(tmp,cv2.COLOR_GRAY2BGR)
pplot(tmp,"normalized depth")

In [ ]:
print(np.max(depth))

In [ ]:
cam.close()

In [1]:
import smbus

In [6]:
bus.close()

In [7]:
(69.81*2 - 1978.19)/ 3

-612.8566666666667